# Step 4 – Case Study: Analyse & catégorisation des reviews (Version Amazon full dataset)

Ce notebook utilise l’ensemble des tables du modèle Amazon (25 tables) pour construire un jeu de données riche de reviews,
puis appliquer les méthodes d’analyse demandées au **Step 4** :

- Prétraitement NLP des reviews
- Clustering non supervisé pour regrouper les avis
- Classification *zero-shot* pour catégoriser automatiquement les reviews
- (Optionnel) classification supervisée si un label est ajouté
- Visualisations et métriques pour le rapport de 5–10 pages

> ℹ️ Les tables utilisées en priorité pour les reviews :
> - `review.csv` : texte des avis + rating
> - `product_reviews.csv` : lien produit ↔ review
> - `product.csv` : informations produit
> - `seller_reviews.csv` : lien vendeur ↔ review
> - `seller.csv` : informations vendeur
>
> Les autres tables (orders, buyer, etc.) peuvent servir à enrichir l’analyse (segmentation par client, période, etc.).

In [ ]:
# ==========================
# 1. Imports des librairies
# ==========================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# modèles avancés
# pip install sentence-transformers transformers
from sentence_transformers import SentenceTransformer
from transformers import pipeline

nltk.download('stopwords')
nltk.download('wordnet')

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.grid'] = True

BASE_PATH = '/mnt/data'  # à adapter si besoin


## 2. Chargement de toutes les tables du modèle

On charge ici les principales tables fournies (25 tables). Adaptable selon ton environnement
(Data Lake, Snowflake, etc.).

In [ ]:
# ==========================
# 2. Load des tables CSV
# ==========================
def load_csv(name: str) -> pd.DataFrame:
    path = os.path.join(BASE_PATH, name)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Fichier introuvable: {path}")
    df = pd.read_csv(path)
    # suppression éventuelle d'une colonne d'index 'Unnamed: 0'
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    return df

buyer            = load_csv('buyer.csv')
carrier          = load_csv('carrier.csv')
cart             = load_csv('cart.csv')
cart_items       = load_csv('cart_items.csv')
category         = load_csv('category.csv')
customer         = load_csv('customer.csv')
customer_payment = load_csv('customer_payment.csv')
customer_shipping= load_csv('customer_shipping.csv')
daily_deals      = load_csv('daily_deals.csv')
discount         = load_csv('discount.csv')
orders           = load_csv('orders.csv')
payment_details  = load_csv('payment_details.csv')
product          = load_csv('product.csv')
product_images   = load_csv('product_images.csv')
product_reviews  = load_csv('product_reviews.csv')
returns          = load_csv('returns.csv')
review           = load_csv('review.csv')
review_images    = load_csv('review_images.csv')
seller           = load_csv('seller.csv')
seller_products  = load_csv('seller_products.csv')
seller_reviews   = load_csv('seller_reviews.csv')
shipment         = load_csv('shipment.csv')
shipping_details = load_csv('shipping_details.csv')
subscription     = load_csv('subscription.csv')
wishlist_item    = load_csv('wishlist_item.csv')

len(buyer), len(review), len(product_reviews), len(product)

## 3. Construction d'une vue analytique `reviews_enriched`

On crée une table analytique qui enrichit chaque review avec :
- les informations produit (`product`)
- la catégorie (`category`)
- le vendeur (`seller`)
- le buyer (client, via `buyer`)

Cette vue servira ensuite pour le NLP et les visualisations.

In [ ]:
# ==========================
# 3. Construction de reviews_enriched
# ==========================
# 1) review + product
reviews_prod = review.merge(product_reviews, on='review_id', how='left')
reviews_prod = reviews_prod.merge(product, on='p_id', how='left')

# 2) ajout catégorie
reviews_prod = reviews_prod.merge(category, on='category_id', how='left', suffixes=('', '_cat'))

# 3) ajout seller
reviews_prod = reviews_prod.merge(seller_reviews, on='review_id', how='left')
reviews_prod = reviews_prod.merge(seller, on='seller_id', how='left', suffixes=('', '_seller'))

# 4) ajout buyer / orders (optionnel, via buyer_id)
reviews_prod = reviews_prod.merge(buyer, on='buyer_id', how='left', suffixes=('', '_buyer'))

reviews_enriched = reviews_prod.copy()

print(reviews_enriched.shape)
reviews_enriched.head()

## 4. Préparation du texte pour le NLP

On crée une variable texte unique à partir de :
- `title`
- `r_desc` (description de la review)
- éventuellement `p_name` (nom produit) pour plus de contexte.


In [ ]:
# ==========================
# 4. Préprocessing texte
# ==========================
text_cols = []
for col in ['title', 'r_desc', 'p_name']:
    if col in reviews_enriched.columns:
        text_cols.append(col)

def build_text(row):
    parts = []
    for c in text_cols:
        val = row.get(c, '')
        if isinstance(val, str):
            parts.append(val)
    return ' '.join(parts)

reviews_enriched['raw_text'] = reviews_enriched.apply(build_text, axis=1)
reviews_enriched['raw_text'] = reviews_enriched['raw_text'].fillna('')

stop = set(stopwords.words('english'))
lemm = WordNetLemmatizer()

def preprocess(text: str) -> str:
    if not isinstance(text, str):
        return ''
    text = text.lower()
    tokens = []
    for w in text.split():
        if w in stop:
            continue
        tokens.append(lemm.lemmatize(w))
    return ' '.join(tokens)

reviews_enriched['clean_text'] = reviews_enriched['raw_text'].apply(preprocess)
reviews_enriched[['raw_text', 'clean_text']].head()

## 5. EDA rapide sur les reviews

- Distribution de la longueur des reviews
- Distribution des ratings
- Aperçu des reviews les plus longues


In [ ]:
# ==========================
# 5. EDA – longueur, rating
# ==========================
df = reviews_enriched.copy()
df['length'] = df['clean_text'].str.split().apply(len)

df['length'].hist(bins=50)
plt.title('Distribution de la longueur des reviews')
plt.xlabel('Nombre de mots')
plt.ylabel('Nombre de reviews')
plt.show()

if 'rating' in df.columns:
    df['rating'].value_counts().sort_index().plot(kind='bar')
    plt.title('Distribution des ratings')
    plt.xlabel('Rating')
    plt.ylabel('Nombre de reviews')
    plt.show()

df[['raw_text', 'length']].sort_values('length', ascending=False).head(5)

## 6. TF-IDF + clustering KMeans

On applique TF-IDF puis KMeans pour obtenir des groupes de reviews.
On calcule le **silhouette score** pour évaluer la qualité des clusters.

In [ ]:
# ==========================
# 6. Clustering KMeans
# ==========================
max_features = 5000
vectorizer = TfidfVectorizer(max_features=max_features)
X_tfidf = vectorizer.fit_transform(df['clean_text'])

k = 6  # nombre de clusters (à ajuster)
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X_tfidf)

sil_score = silhouette_score(X_tfidf, df['cluster'])
print(f"Silhouette score (K={k}): {sil_score:.3f}")

df['cluster'].value_counts().sort_index().plot(kind='bar')
plt.title('Répartition des clusters')
plt.xlabel('Cluster')
plt.ylabel('Nombre de reviews')
plt.show()

df.groupby('cluster')['clean_text'].apply(lambda x: ' | '.join(x.iloc[:3])).head()

## 7. Zero-shot classification

On utilise un modèle pré-entraîné (`facebook/bart-large-mnli`) pour attribuer
un label à chaque review parmi une liste de catégories métier.

Exemple de labels (à adapter à ton cas) :
- `delivery issue`
- `product quality`
- `price problem`
- `customer service`
- `excellent overall`
- `useless review`


In [ ]:
# ==========================
# 7. Zero-shot classification
# ==========================
zero_shot_classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

candidate_labels = [
    'delivery issue',
    'product quality',
    'price problem',
    'customer service',
    'excellent overall',
    'useless review',
]

def classify_zero_shot(text: str) -> str:
    if not isinstance(text, str) or text.strip() == '':
        return 'unknown'
    result = zero_shot_classifier(text, candidate_labels=candidate_labels)
    return result['labels'][0]

# pour contrôler la durée de calcul : échantillon
sample_size = min(1000, len(df))
df_sample = df.sample(sample_size, random_state=42).copy()

df_sample['zero_shot_label'] = df_sample['raw_text'].apply(classify_zero_shot)

df_sample['zero_shot_label'].value_counts().plot(kind='bar')
plt.title('Répartition des labels zero-shot (échantillon)')
plt.xlabel('Label')
plt.ylabel('Nombre de reviews')
plt.show()

df_sample[['raw_text', 'zero_shot_label']].head(10)

## 8. Classification supervisée (optionnel)

Si tu crées une colonne `label` (par exemple 1 = review utile, 0 = inutile),
tu peux entraîner un modèle simple Logistic Regression pour évaluer les performances
d’un classifieur supervisé.


In [ ]:
# ==========================
# 8. Classification supervisée (si df contient 'label')
# ==========================
if 'label' in df.columns:
    print("Colonne 'label' trouvée, entraînement d'un modèle supervisé...")
    X = X_tfidf
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print('\nClassification report:')
    print(classification_report(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred)
    print('\nConfusion matrix:\n', cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title('Matrice de confusion')
    plt.colorbar(im)
    plt.show()
else:
    print("Aucune colonne 'label' trouvée. Ajoute un label manuellement si tu veux tester la partie supervisée.")

## 9. Embeddings (Sentence Transformers) + PCA (optionnel avancé)

On encode les reviews avec un modèle type `all-MiniLM-L6-v2`, puis on visualise
les embeddings projetés en 2D à l’aide de PCA.

In [ ]:
# ==========================
# 9. Embeddings + PCA
# ==========================
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
st_model = SentenceTransformer(model_name)

sample_size = min(1500, len(df))
df_emb = df.sample(sample_size, random_state=42).copy()

embeddings = st_model.encode(df_emb['clean_text'].tolist(), show_progress_bar=True)

pca = PCA(n_components=2, random_state=42)
coords = pca.fit_transform(embeddings)

df_emb['x'] = coords[:, 0]
df_emb['y'] = coords[:, 1]

plt.scatter(df_emb['x'], df_emb['y'], c=df_emb.get('cluster', 0), alpha=0.6)
plt.title('Projection PCA des embeddings de reviews')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

## 10. Synthèse & export pour le rapport

À partir de ce notebook, tu peux :
- extraire des KPIs :
  - nombre de reviews par rating, par catégorie produit, par vendeur
  - répartition des clusters, des labels zero-shot
- exporter des jeux de données agrégés vers Snowflake / ton Data Warehouse
- faire des captures d’écran des graphes pour le rapport et la présentation.

Tu peux par exemple créer une vue agrégée par produit + catégorie zero-shot :

In [ ]:
# Exemple d'agrégation (à adapter)
if 'zero_shot_label' in df_sample.columns:
    agg = (
        df_sample
        .groupby(['p_id', 'zero_shot_label'])
        .size()
        .reset_index(name='nb_reviews')
    )
    agg.head()
else:
    print("L'agrégation par zero_shot_label nécessite d'avoir exécuté la cellule de zero-shot.")